In [1]:
%%time
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
plt.style.use('seaborn')
pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 10000)

Wall time: 2 s


In [2]:
df_train = pd.read_csv('Printer--training.csv')
df_test = pd.read_csv('Printer--test.csv')
df_train["Completion Time"] = 0
df_train.replace(0,np.NaN, inplace=True)
df_train.drop(['case description', 'event org:resource'], axis=1)

,eventID,case concept:name,case Class,event concept:name,event lifecycle:transition,event time:timestamp,Completion Time
0,NaN,1,Print,Job,start,01-01-1970 01:00:00.000,NaN
1,1.000000e+00,1,Print,Remote Print,complete,01-01-1970 01:15:00.000,NaN
2,2.000000e+00,1,Print,Read Print Options,complete,01-01-1970 01:26:00.000,NaN
3,3.000000e+00,1,Print,Rasterization,start,01-01-1970 01:38:00.000,NaN
4,4.000000e+00,1,Print,Interpretation,start,01-01-1970 01:51:00.000,NaN
...,...,...,...,...,...,...,...
36623,4.294967e+10,18,Print,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000,NaN
36624,4.294967e+10,18,Print,Fusing,complete,25-12-1970 16:05:00.000,NaN
36625,4.294967e+10,18,Print,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000,NaN
36626,4.294967e+10,18,Print,Erase Charge on Drum,complete,25-12-1970 16:22:00.000,NaN


In [3]:
def EventTime(data):
    memory = {} # Stores active events, key=event, value=index of start
    for i, j in data[["event concept:name"]].iterrows():
        
        if j[0] not in memory.keys() and data.loc[i]["event lifecycle:transition"] == "start":
            memory[j[0]] = i
        
        elif j[0] in memory.keys():
            time = pd.to_datetime(data.loc[i]["event time:timestamp"], dayfirst=True) - pd.to_datetime(data.loc[memory[j[0]]]["event time:timestamp"], dayfirst=True)           # print(time)
            data.loc[i,"Completion Time"] = time
            del memory[j[0]] # Removes the completed event from active event dictionary(memory)
    
    return data # Returns a new dataframe with event completion time. Can be improved.


In [4]:
def AverageTime(df_train):
    """
    1. Computes the event completion time for events that don't have a "start" by taking the 
       difference between said event's completion time and the previous (i-1) event time. Still have 
       to determine how correct this is, as several events can run in parallel.
    2. Sets new index on df and computes the remaining event times (for events that have a start and completion time)
       by running the EventTime function case by case. Can be improved.
    3. Returns average completion time per event.
    
    """
    
    for i in range(1, df_train['Completion Time'].shape[0]-1): # 1
        if pd.isnull(df_train['Completion Time'][i]):
            if df_train['event lifecycle:transition'][i] == 'complete':
                time = pd.to_datetime(df_train["event time:timestamp"][i], dayfirst=True) - pd.to_datetime(df_train["event time:timestamp"][i-1], dayfirst=True)          
                df_train['Completion Time'][i] = time
    
    df_train.set_index(["case concept:name", "eventID "], inplace=True) # 2
    
    for idx, new_df in df_train.groupby(level=0): # Adds event completion time, case by case.
        df_train.loc[idx] = EventTime(new_df)
        
    Average_time = df_train.groupby(['event concept:name'])[['Completion Time']].mean() # 3
    return Average_time

In [5]:
%%time
AverageTime(df_train)

<ipython-input-4-979874e91603>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Completion Time'][i] = time


Wall time: 1min 10s


,Completion Time
event concept:name,
A/D Conversion,0 days 00:14:59.642857142
AM Screening,0 days 00:11:01.022727272
Accumulate Images,0 days 00:05:44.285714285
Apply Heat,0 days 00:07:42.572614107
Apply Negative Charge on Drum,0 days 00:12:19.428571428
Apply Pressure,0 days 00:07:21.161825726
Calc Quantization Error,0 days 00:09:07.261146496
Calc Total Neighbor Quant Error,0 days 00:06:45.095541401
Coat Light Toner on Drum,0 days 00:14:54.572127139
